In [1]:
import os
import hdf5storage
import numpy as np
import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.feature_selection import f_classif
# from sklearn.decomposition import PCA
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, GRU, LSTM, Dense
# from tensorflow.keras.optimizers import Adam
# from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, auc
# from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, PrecisionRecallDisplay

# import seaborn as sns
# import matplotlib.pyplot as plt
#
# from sklearn.preprocessing import MinMaxScaler

import pandas as pd
from pgmpy.estimators import PC
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator

from scipy.stats import entropy

import os
import pandas as pd
from scipy.io import loadmat

import networkx as nx
import matplotlib.pyplot as plt

In [2]:
"""
Load Data Seperate All data in Protective and Non-Prtective Data
"""

# Assuming the 'Data' folder is in the current working directory
data_folders = ['./Data']

# Initialize lists to store the separated dataframes
protective_dfs = []
non_protective_dfs = []

# Loop through each data folder
for data_folder in data_folders:
    # List all .mat files in the current data folder
    mat_files = [f for f in os.listdir(data_folder) if f.endswith('.mat')]
    # Load each mat file
    for mat_file in mat_files:
        # Construct the full path to the .mat file
        mat_path = os.path.join(data_folder, mat_file)
        # Load the .mat file
        mat_data = loadmat(mat_path)
        # Convert the data into a pandas dataframe
        df = pd.DataFrame(mat_data['data'])
        # Select only the first 70 columns and the last column (73rd) which contains the behavior label
        df = df.iloc[:, list(range(66)) + [70] + [72]]
        # Split the data based on the protective behavior label
        # Assuming the last column in df is the protective behavior label
        protective_behavior = df.iloc[:, -1]
        protective_df = df[protective_behavior == 1]
        non_protective_df = df[protective_behavior == 0]
        # Append the resulting dataframes to their respective lists
        protective_dfs.append(protective_df)
        non_protective_dfs.append(non_protective_df)

# Concatenate all protective and non-protective dataframes
all_protective_data = pd.concat(protective_dfs, axis=0, ignore_index=True)
all_non_protective_data = pd.concat(non_protective_dfs, axis=0, ignore_index=True)

# Now `all_protective_data` and `all_non_protective_data` hold the protective and non-protective data respectively
# You can process these dataframes as needed for your analysis or save them to new .mat files
print(all_protective_data.shape)
print(all_non_protective_data.shape)

(77298, 68)
(437247, 68)


In [3]:
nodes_data = {}

# There are 22 nodes, so we loop through each
for node in range(1, 23):
    # Calculate the index for x, y, and z based on the node number
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44

    # Extract the data for the current node
    node_data = all_protective_data.iloc[:, [x_index, y_index, z_index]]

    # Assign the node data to the corresponding entry in the dictionary
    nodes_data[f'node_{node}'] = node_data

In [4]:
 # (node_from, node_to) based on the known graph structure
edges = [
    (1, 2), (1, 5), (1, 8), (2, 3), (3, 4), (5, 6), (6, 7),
    (8, 9), (9, 10), (9, 15), (9, 20), (10, 11), (11, 12),
    (12, 13), (13, 14), (15, 16), (16, 17), (17, 18), (18, 19),
    (9, 20), (20, 21), (21, 22)
]

In [5]:
def calculate_kl_divergence(p, q, epsilon=1e-10):
    p_normed = (p + epsilon) / (np.sum(p) + epsilon * len(p))
    q_normed = (q + epsilon) / (np.sum(q) + epsilon * len(q))
    return entropy(p_normed, q_normed)

# Exercise Type Analysis

In [6]:
exercise_labels = {
    1: "One-leg-stand",
    2: "Reach-forward",
    3: "Bend",
    4: "Sit-to-stand",
    5: "Stand-to-sit",
    6: "Sitting still",
    7: "Standing still",
    8: "Walking",
    0: "Others"
}
def categorize_exercises(dataframe):
    # Assuming the penultimate column in dataframe is the Exercise Type
    exercise_type = dataframe.iloc[:, -2]
    categorized_data = {}
    for label, description in exercise_labels.items():
        # Filter the dataframe by each exercise type
        exercise_df = dataframe[exercise_type == label]
        # Store the filtered dataframe in a dictionary
        categorized_data[description] = exercise_df
    return categorized_data

categorized_protective = categorize_exercises(all_protective_data)
categorized_non_protective = categorize_exercises(all_non_protective_data)

# Bend

In [7]:
nodes_data = {}

for node in range(1, 23):
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44
    node_data = categorized_protective["Bend"].iloc[:, [x_index, y_index, z_index]]
    nodes_data[f'node_{node}'] = node_data

In [8]:
def analyze_joint_interactions(node_data_1, node_data_2, labels=('node_1', 'node_2')):
    """Analyzes interactions between two nodes and calculates weight based on the direction of influence."""
    directions = []
    kl_divergences = {f'{labels[0]}_to_{labels[1]}': [], f'{labels[1]}_to_{labels[0]}': []}

    for i in range(3):  # For X, Y, Z coordinates
        df = pd.DataFrame({
            labels[0]: node_data_1[i].iloc[:6113],
            labels[1]: node_data_2[i].iloc[:6113]
        })

        pc = PC(df)
        estimated_dag = pc.estimate()
        bn = BayesianModel(estimated_dag.edges())
        bn.fit(df, estimator=MaximumLikelihoodEstimator)

        cpd_for_node_1 = bn.get_cpds(node=labels[0])
        cpd_for_node_2 = bn.get_cpds(node=labels[1])

        p_node_1_given_2 = cpd_for_node_1.values
        p_node_2_given_1 = cpd_for_node_2.values

        kl_divergence_1_to_2 = np.mean(calculate_kl_divergence(p_node_1_given_2, p_node_2_given_1))
        kl_divergence_2_to_1 = np.mean(calculate_kl_divergence(p_node_2_given_1, p_node_1_given_2))

        direction_key = f'{labels[0]}_to_{labels[1]}' if kl_divergence_1_to_2 < kl_divergence_2_to_1 else f'{labels[1]}_to_{labels[0]}'
        directions.append(direction_key)
        kl_divergences[direction_key].append(min(kl_divergence_1_to_2, kl_divergence_2_to_1))

    final_direction = max(set(directions), key=directions.count)
    final_weight = np.mean(kl_divergences[final_direction])

    return final_direction, final_weight

In [9]:
node_1_data = [nodes_data['node_1'][0], nodes_data['node_1'][22], nodes_data['node_1'][44]]
node_2_data = [nodes_data['node_2'][1], nodes_data['node_2'][23], nodes_data['node_2'][45]]
node_3_data = [nodes_data['node_3'][2], nodes_data['node_3'][24], nodes_data['node_3'][46]]
node_4_data = [nodes_data['node_4'][3], nodes_data['node_4'][25], nodes_data['node_4'][47]]
node_5_data = [nodes_data['node_5'][4], nodes_data['node_5'][26], nodes_data['node_5'][48]]
node_6_data = [nodes_data['node_6'][5], nodes_data['node_6'][27], nodes_data['node_6'][49]]
node_7_data = [nodes_data['node_7'][6], nodes_data['node_7'][28], nodes_data['node_7'][50]]
node_8_data = [nodes_data['node_8'][7], nodes_data['node_8'][29], nodes_data['node_8'][51]]
node_9_data = [nodes_data['node_9'][8], nodes_data['node_9'][30], nodes_data['node_9'][52]]
node_10_data = [nodes_data['node_10'][9], nodes_data['node_10'][31], nodes_data['node_10'][53]]
node_11_data = [nodes_data['node_11'][10], nodes_data['node_11'][32], nodes_data['node_11'][54]]
node_12_data = [nodes_data['node_12'][11], nodes_data['node_12'][33], nodes_data['node_12'][55]]
node_13_data = [nodes_data['node_13'][12], nodes_data['node_13'][34], nodes_data['node_13'][56]]
node_14_data = [nodes_data['node_14'][13], nodes_data['node_14'][35], nodes_data['node_14'][57]]
node_15_data = [nodes_data['node_15'][14], nodes_data['node_15'][36], nodes_data['node_15'][58]]
node_16_data = [nodes_data['node_16'][15], nodes_data['node_16'][37], nodes_data['node_16'][59]]
node_17_data = [nodes_data['node_17'][16], nodes_data['node_17'][38], nodes_data['node_17'][60]]
node_18_data = [nodes_data['node_18'][17], nodes_data['node_18'][39], nodes_data['node_18'][61]]
node_19_data = [nodes_data['node_19'][18], nodes_data['node_19'][40], nodes_data['node_19'][62]]
node_20_data = [nodes_data['node_20'][19], nodes_data['node_20'][41], nodes_data['node_20'][63]]
node_21_data = [nodes_data['node_21'][20], nodes_data['node_21'][42], nodes_data['node_21'][64]]
node_22_data = [nodes_data['node_22'][21], nodes_data['node_22'][43], nodes_data['node_22'][65]]

In [10]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_2_data, ('node_1', 'node_2'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_1, Final weight: 8.708239025255935


In [11]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_5_data, ('node_1', 'node_5'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_5_to_node_1, Final weight: 8.708239025255935


In [12]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_8_data, ('node_1', 'node_8'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_1, Final weight: 8.708239025255935


In [13]:
final_direction, final_weight = analyze_joint_interactions(node_2_data, node_3_data, ('node_2', 'node_3'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_3_to_node_2, Final weight: 8.710923529717265


In [14]:
final_direction, final_weight = analyze_joint_interactions(node_3_data, node_4_data, ('node_3', 'node_4'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_4_to_node_3, Final weight: 8.711530836060694


In [15]:
final_direction, final_weight = analyze_joint_interactions(node_5_data, node_6_data, ('node_5', 'node_6'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_5, Final weight: 8.71113283770636


In [16]:
final_direction, final_weight = analyze_joint_interactions(node_6_data, node_7_data, ('node_6', 'node_7'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_7, Final weight: 8.711697977701363


In [17]:
final_direction, final_weight = analyze_joint_interactions(node_8_data, node_9_data, ('node_8', 'node_9'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_8, Final weight: 8.71089531348253


In [18]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_10_data, ('node_9', 'node_10'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_10, Final weight: 8.71114620055997


In [19]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_15_data, ('node_9', 'node_15'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_15, Final weight: 8.71114620055997


In [20]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_20_data, ('node_9', 'node_20'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_20_to_node_9, Final weight: 8.71164915749699


In [21]:
final_direction, final_weight = analyze_joint_interactions(node_10_data, node_11_data, ('node_10', 'node_11'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_10_to_node_11, Final weight: 8.712467199072405


In [22]:
final_direction, final_weight = analyze_joint_interactions(node_11_data, node_12_data, ('node_11', 'node_12'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_11, Final weight: 8.715296099946494


In [23]:
final_direction, final_weight = analyze_joint_interactions(node_12_data, node_13_data, ('node_12', 'node_13'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_13, Final weight: 8.716247229225123


In [24]:
final_direction, final_weight = analyze_joint_interactions(node_13_data, node_14_data, ('node_13', 'node_14'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [25]:
final_direction, final_weight = analyze_joint_interactions(node_15_data, node_16_data, ('node_15', 'node_16'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_16, Final weight: 8.712467199072757


In [26]:
final_direction, final_weight = analyze_joint_interactions(node_16_data, node_17_data, ('node_16', 'node_17'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_16, Final weight: 8.714769797592512


In [27]:
final_direction, final_weight = analyze_joint_interactions(node_17_data, node_18_data, ('node_17', 'node_18'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_18, Final weight: 8.715973295794479


In [28]:
final_direction, final_weight = analyze_joint_interactions(node_18_data, node_19_data, ('node_18', 'node_19'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [29]:
final_direction, final_weight = analyze_joint_interactions(node_20_data, node_21_data, ('node_20', 'node_21'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_20, Final weight: 8.71492264326446


In [30]:
final_direction, final_weight = analyze_joint_interactions(node_21_data, node_22_data, ('node_21', 'node_22'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

# Sit to Stand

In [31]:
nodes_data = {}

for node in range(1, 23):
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44
    node_data = categorized_protective["Sit-to-stand"].iloc[:, [x_index, y_index, z_index]]
    nodes_data[f'node_{node}'] = node_data

def analyze_joint_interactions(node_data_1, node_data_2, labels=('node_1', 'node_2')):
    """Analyzes interactions between two nodes and calculates weight based on the direction of influence."""
    directions = []
    kl_divergences = {f'{labels[0]}_to_{labels[1]}': [], f'{labels[1]}_to_{labels[0]}': []}

    for i in range(3):  # For X, Y, Z coordinates
        df = pd.DataFrame({
            labels[0]: node_data_1[i].iloc[:7735],
            labels[1]: node_data_2[i].iloc[:7735]
        })

        pc = PC(df)
        estimated_dag = pc.estimate()
        bn = BayesianModel(estimated_dag.edges())
        bn.fit(df, estimator=MaximumLikelihoodEstimator)

        cpd_for_node_1 = bn.get_cpds(node=labels[0])
        cpd_for_node_2 = bn.get_cpds(node=labels[1])

        p_node_1_given_2 = cpd_for_node_1.values
        p_node_2_given_1 = cpd_for_node_2.values

        kl_divergence_1_to_2 = np.mean(calculate_kl_divergence(p_node_1_given_2, p_node_2_given_1))
        kl_divergence_2_to_1 = np.mean(calculate_kl_divergence(p_node_2_given_1, p_node_1_given_2))

        direction_key = f'{labels[0]}_to_{labels[1]}' if kl_divergence_1_to_2 < kl_divergence_2_to_1 else f'{labels[1]}_to_{labels[0]}'
        directions.append(direction_key)
        kl_divergences[direction_key].append(min(kl_divergence_1_to_2, kl_divergence_2_to_1))

    final_direction = max(set(directions), key=directions.count)
    final_weight = np.mean(kl_divergences[final_direction])

    return final_direction, final_weight

In [32]:
node_1_data = [nodes_data['node_1'][0], nodes_data['node_1'][22], nodes_data['node_1'][44]]
node_2_data = [nodes_data['node_2'][1], nodes_data['node_2'][23], nodes_data['node_2'][45]]
node_3_data = [nodes_data['node_3'][2], nodes_data['node_3'][24], nodes_data['node_3'][46]]
node_4_data = [nodes_data['node_4'][3], nodes_data['node_4'][25], nodes_data['node_4'][47]]
node_5_data = [nodes_data['node_5'][4], nodes_data['node_5'][26], nodes_data['node_5'][48]]
node_6_data = [nodes_data['node_6'][5], nodes_data['node_6'][27], nodes_data['node_6'][49]]
node_7_data = [nodes_data['node_7'][6], nodes_data['node_7'][28], nodes_data['node_7'][50]]
node_8_data = [nodes_data['node_8'][7], nodes_data['node_8'][29], nodes_data['node_8'][51]]
node_9_data = [nodes_data['node_9'][8], nodes_data['node_9'][30], nodes_data['node_9'][52]]
node_10_data = [nodes_data['node_10'][9], nodes_data['node_10'][31], nodes_data['node_10'][53]]
node_11_data = [nodes_data['node_11'][10], nodes_data['node_11'][32], nodes_data['node_11'][54]]
node_12_data = [nodes_data['node_12'][11], nodes_data['node_12'][33], nodes_data['node_12'][55]]
node_13_data = [nodes_data['node_13'][12], nodes_data['node_13'][34], nodes_data['node_13'][56]]
node_14_data = [nodes_data['node_14'][13], nodes_data['node_14'][35], nodes_data['node_14'][57]]
node_15_data = [nodes_data['node_15'][14], nodes_data['node_15'][36], nodes_data['node_15'][58]]
node_16_data = [nodes_data['node_16'][15], nodes_data['node_16'][37], nodes_data['node_16'][59]]
node_17_data = [nodes_data['node_17'][16], nodes_data['node_17'][38], nodes_data['node_17'][60]]
node_18_data = [nodes_data['node_18'][17], nodes_data['node_18'][39], nodes_data['node_18'][61]]
node_19_data = [nodes_data['node_19'][18], nodes_data['node_19'][40], nodes_data['node_19'][62]]
node_20_data = [nodes_data['node_20'][19], nodes_data['node_20'][41], nodes_data['node_20'][63]]
node_21_data = [nodes_data['node_21'][20], nodes_data['node_21'][42], nodes_data['node_21'][64]]
node_22_data = [nodes_data['node_22'][21], nodes_data['node_22'][43], nodes_data['node_22'][65]]

In [33]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_2_data, ('node_1', 'node_2'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_1, Final weight: 8.95133994345782


In [34]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_5_data, ('node_1', 'node_5'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_5_to_node_1, Final weight: 8.95133994345782


In [35]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_8_data, ('node_1', 'node_8'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_1, Final weight: 8.95133994345782


In [36]:
final_direction, final_weight = analyze_joint_interactions(node_2_data, node_3_data, ('node_2', 'node_3'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_3_to_node_2, Final weight: 8.952111378456136


In [37]:
final_direction, final_weight = analyze_joint_interactions(node_3_data, node_4_data, ('node_3', 'node_4'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_4_to_node_3, Final weight: 8.952500423503542


In [38]:
final_direction, final_weight = analyze_joint_interactions(node_5_data, node_6_data, ('node_5', 'node_6'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_5, Final weight: 8.951992317780235


In [39]:
final_direction, final_weight = analyze_joint_interactions(node_6_data, node_7_data, ('node_6', 'node_7'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_7, Final weight: 8.952112016794958


In [40]:
final_direction, final_weight = analyze_joint_interactions(node_8_data, node_9_data, ('node_8', 'node_9'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_8, Final weight: 8.951939347471447


In [41]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_10_data, ('node_9', 'node_10'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_10, Final weight: 8.951939347470384


In [42]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_15_data, ('node_9', 'node_15'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_15, Final weight: 8.951939347470384


In [43]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_20_data, ('node_9', 'node_20'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_20_to_node_9, Final weight: 8.952084931470901


In [44]:
final_direction, final_weight = analyze_joint_interactions(node_10_data, node_11_data, ('node_10', 'node_11'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_10_to_node_11, Final weight: 8.952414080504129


In [45]:
final_direction, final_weight = analyze_joint_interactions(node_11_data, node_12_data, ('node_11', 'node_12'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_11, Final weight: 8.952599902648304


In [46]:
final_direction, final_weight = analyze_joint_interactions(node_12_data, node_13_data, ('node_12', 'node_13'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_13, Final weight: 8.952629823508595


In [47]:
final_direction, final_weight = analyze_joint_interactions(node_13_data, node_14_data, ('node_13', 'node_14'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_13_to_node_14, Final weight: 8.952629823508593


In [48]:
final_direction, final_weight = analyze_joint_interactions(node_15_data, node_16_data, ('node_15', 'node_16'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_16, Final weight: 8.952414080502521


In [49]:
final_direction, final_weight = analyze_joint_interactions(node_16_data, node_17_data, ('node_16', 'node_17'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_16, Final weight: 8.952513665757037


In [50]:
final_direction, final_weight = analyze_joint_interactions(node_17_data, node_18_data, ('node_17', 'node_18'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_18, Final weight: 8.952543586617326


In [51]:
final_direction, final_weight = analyze_joint_interactions(node_18_data, node_19_data, ('node_18', 'node_19'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_19_to_node_18, Final weight: 8.952543586617326


In [52]:
final_direction, final_weight = analyze_joint_interactions(node_20_data, node_21_data, ('node_20', 'node_21'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_20, Final weight: 8.952427326185513


In [53]:
final_direction, final_weight = analyze_joint_interactions(node_21_data, node_22_data, ('node_21', 'node_22'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_22, Final weight: 8.95245725479766


# Stand to Sit

In [54]:
nodes_data = {}

for node in range(1, 23):
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44
    node_data = categorized_protective["Stand-to-sit"].iloc[:, [x_index, y_index, z_index]]
    nodes_data[f'node_{node}'] = node_data

def analyze_joint_interactions(node_data_1, node_data_2, labels=('node_1', 'node_2')):
    """Analyzes interactions between two nodes and calculates weight based on the direction of influence."""
    directions = []
    kl_divergences = {f'{labels[0]}_to_{labels[1]}': [], f'{labels[1]}_to_{labels[0]}': []}

    for i in range(3):  # For X, Y, Z coordinates
        df = pd.DataFrame({
            labels[0]: node_data_1[i].iloc[:10872],
            labels[1]: node_data_2[i].iloc[:10872]
        })

        pc = PC(df)
        estimated_dag = pc.estimate()
        bn = BayesianModel(estimated_dag.edges())
        bn.fit(df, estimator=MaximumLikelihoodEstimator)

        cpd_for_node_1 = bn.get_cpds(node=labels[0])
        cpd_for_node_2 = bn.get_cpds(node=labels[1])

        p_node_1_given_2 = cpd_for_node_1.values
        p_node_2_given_1 = cpd_for_node_2.values

        kl_divergence_1_to_2 = np.mean(calculate_kl_divergence(p_node_1_given_2, p_node_2_given_1))
        kl_divergence_2_to_1 = np.mean(calculate_kl_divergence(p_node_2_given_1, p_node_1_given_2))

        direction_key = f'{labels[0]}_to_{labels[1]}' if kl_divergence_1_to_2 < kl_divergence_2_to_1 else f'{labels[1]}_to_{labels[0]}'
        directions.append(direction_key)
        kl_divergences[direction_key].append(min(kl_divergence_1_to_2, kl_divergence_2_to_1))

    final_direction = max(set(directions), key=directions.count)
    final_weight = np.mean(kl_divergences[final_direction])

    return final_direction, final_weight

In [55]:
node_1_data = [nodes_data['node_1'][0], nodes_data['node_1'][22], nodes_data['node_1'][44]]
node_2_data = [nodes_data['node_2'][1], nodes_data['node_2'][23], nodes_data['node_2'][45]]
node_3_data = [nodes_data['node_3'][2], nodes_data['node_3'][24], nodes_data['node_3'][46]]
node_4_data = [nodes_data['node_4'][3], nodes_data['node_4'][25], nodes_data['node_4'][47]]
node_5_data = [nodes_data['node_5'][4], nodes_data['node_5'][26], nodes_data['node_5'][48]]
node_6_data = [nodes_data['node_6'][5], nodes_data['node_6'][27], nodes_data['node_6'][49]]
node_7_data = [nodes_data['node_7'][6], nodes_data['node_7'][28], nodes_data['node_7'][50]]
node_8_data = [nodes_data['node_8'][7], nodes_data['node_8'][29], nodes_data['node_8'][51]]
node_9_data = [nodes_data['node_9'][8], nodes_data['node_9'][30], nodes_data['node_9'][52]]
node_10_data = [nodes_data['node_10'][9], nodes_data['node_10'][31], nodes_data['node_10'][53]]
node_11_data = [nodes_data['node_11'][10], nodes_data['node_11'][32], nodes_data['node_11'][54]]
node_12_data = [nodes_data['node_12'][11], nodes_data['node_12'][33], nodes_data['node_12'][55]]
node_13_data = [nodes_data['node_13'][12], nodes_data['node_13'][34], nodes_data['node_13'][56]]
node_14_data = [nodes_data['node_14'][13], nodes_data['node_14'][35], nodes_data['node_14'][57]]
node_15_data = [nodes_data['node_15'][14], nodes_data['node_15'][36], nodes_data['node_15'][58]]
node_16_data = [nodes_data['node_16'][15], nodes_data['node_16'][37], nodes_data['node_16'][59]]
node_17_data = [nodes_data['node_17'][16], nodes_data['node_17'][38], nodes_data['node_17'][60]]
node_18_data = [nodes_data['node_18'][17], nodes_data['node_18'][39], nodes_data['node_18'][61]]
node_19_data = [nodes_data['node_19'][18], nodes_data['node_19'][40], nodes_data['node_19'][62]]
node_20_data = [nodes_data['node_20'][19], nodes_data['node_20'][41], nodes_data['node_20'][63]]
node_21_data = [nodes_data['node_21'][20], nodes_data['node_21'][42], nodes_data['node_21'][64]]
node_22_data = [nodes_data['node_22'][21], nodes_data['node_22'][43], nodes_data['node_22'][65]]

In [56]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_2_data, ('node_1', 'node_2'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_1, Final weight: 9.291209258958043


In [57]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_5_data, ('node_1', 'node_5'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_5_to_node_1, Final weight: 9.291209258958043


In [58]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_8_data, ('node_1', 'node_8'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_1, Final weight: 9.291209258958043


In [59]:
final_direction, final_weight = analyze_joint_interactions(node_2_data, node_3_data, ('node_2', 'node_3'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_3_to_node_2, Final weight: 9.291853585810992


In [60]:
final_direction, final_weight = analyze_joint_interactions(node_3_data, node_4_data, ('node_3', 'node_4'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_4_to_node_3, Final weight: 9.292231927541348


In [61]:
final_direction, final_weight = analyze_joint_interactions(node_5_data, node_6_data, ('node_5', 'node_6'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_5, Final weight: 9.291697641437707


In [62]:
final_direction, final_weight = analyze_joint_interactions(node_6_data, node_7_data, ('node_6', 'node_7'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_7, Final weight: 9.291740297915672


In [63]:
final_direction, final_weight = analyze_joint_interactions(node_8_data, node_9_data, ('node_8', 'node_9'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_8, Final weight: 9.291716482611717


In [64]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_10_data, ('node_9', 'node_10'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_10, Final weight: 9.291801709115491


In [65]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_15_data, ('node_9', 'node_15'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_15, Final weight: 9.291801709115491


In [66]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_20_data, ('node_9', 'node_20'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_20_to_node_9, Final weight: 9.291943039180953


In [67]:
final_direction, final_weight = analyze_joint_interactions(node_10_data, node_11_data, ('node_10', 'node_11'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_10_to_node_11, Final weight: 9.292262573701144


In [68]:
final_direction, final_weight = analyze_joint_interactions(node_11_data, node_12_data, ('node_11', 'node_12'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_11, Final weight: 9.292416175368787


In [69]:
final_direction, final_weight = analyze_joint_interactions(node_12_data, node_13_data, ('node_12', 'node_13'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_13, Final weight: 9.292416175368787


In [70]:
final_direction, final_weight = analyze_joint_interactions(node_13_data, node_14_data, ('node_13', 'node_14'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_13_to_node_14, Final weight: 9.292416175368787


In [71]:
final_direction, final_weight = analyze_joint_interactions(node_15_data, node_16_data, ('node_15', 'node_16'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_16, Final weight: 9.292262573702027


In [72]:
final_direction, final_weight = analyze_joint_interactions(node_16_data, node_17_data, ('node_16', 'node_17'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_16, Final weight: 9.29247516845461


In [73]:
final_direction, final_weight = analyze_joint_interactions(node_17_data, node_18_data, ('node_17', 'node_18'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_18, Final weight: 9.292539017639784


In [74]:
final_direction, final_weight = analyze_joint_interactions(node_18_data, node_19_data, ('node_18', 'node_19'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_19_to_node_18, Final weight: 9.292539017639783


In [75]:
final_direction, final_weight = analyze_joint_interactions(node_20_data, node_21_data, ('node_20', 'node_21'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_20, Final weight: 9.292272005586975


In [76]:
final_direction, final_weight = analyze_joint_interactions(node_21_data, node_22_data, ('node_21', 'node_22'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_22, Final weight: 9.292293316118625


# Sitting Still

In [77]:
nodes_data = {}

for node in range(1, 23):
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44
    node_data = categorized_protective["Sitting still"].iloc[:, [x_index, y_index, z_index]]
    nodes_data[f'node_{node}'] = node_data

def analyze_joint_interactions(node_data_1, node_data_2, labels=('node_1', 'node_2')):
    """Analyzes interactions between two nodes and calculates weight based on the direction of influence."""
    directions = []
    kl_divergences = {f'{labels[0]}_to_{labels[1]}': [], f'{labels[1]}_to_{labels[0]}': []}

    for i in range(3):  # For X, Y, Z coordinates
        df = pd.DataFrame({
            labels[0]: node_data_1[i].iloc[:3968],
            labels[1]: node_data_2[i].iloc[:3968]
        })

        pc = PC(df)
        estimated_dag = pc.estimate()
        bn = BayesianModel(estimated_dag.edges())
        bn.fit(df, estimator=MaximumLikelihoodEstimator)

        cpd_for_node_1 = bn.get_cpds(node=labels[0])
        cpd_for_node_2 = bn.get_cpds(node=labels[1])

        p_node_1_given_2 = cpd_for_node_1.values
        p_node_2_given_1 = cpd_for_node_2.values

        kl_divergence_1_to_2 = np.mean(calculate_kl_divergence(p_node_1_given_2, p_node_2_given_1))
        kl_divergence_2_to_1 = np.mean(calculate_kl_divergence(p_node_2_given_1, p_node_1_given_2))

        direction_key = f'{labels[0]}_to_{labels[1]}' if kl_divergence_1_to_2 < kl_divergence_2_to_1 else f'{labels[1]}_to_{labels[0]}'
        directions.append(direction_key)
        kl_divergences[direction_key].append(min(kl_divergence_1_to_2, kl_divergence_2_to_1))

    final_direction = max(set(directions), key=directions.count)
    final_weight = np.mean(kl_divergences[final_direction])

    return final_direction, final_weight

In [78]:
node_1_data = [nodes_data['node_1'][0], nodes_data['node_1'][22], nodes_data['node_1'][44]]
node_2_data = [nodes_data['node_2'][1], nodes_data['node_2'][23], nodes_data['node_2'][45]]
node_3_data = [nodes_data['node_3'][2], nodes_data['node_3'][24], nodes_data['node_3'][46]]
node_4_data = [nodes_data['node_4'][3], nodes_data['node_4'][25], nodes_data['node_4'][47]]
node_5_data = [nodes_data['node_5'][4], nodes_data['node_5'][26], nodes_data['node_5'][48]]
node_6_data = [nodes_data['node_6'][5], nodes_data['node_6'][27], nodes_data['node_6'][49]]
node_7_data = [nodes_data['node_7'][6], nodes_data['node_7'][28], nodes_data['node_7'][50]]
node_8_data = [nodes_data['node_8'][7], nodes_data['node_8'][29], nodes_data['node_8'][51]]
node_9_data = [nodes_data['node_9'][8], nodes_data['node_9'][30], nodes_data['node_9'][52]]
node_10_data = [nodes_data['node_10'][9], nodes_data['node_10'][31], nodes_data['node_10'][53]]
node_11_data = [nodes_data['node_11'][10], nodes_data['node_11'][32], nodes_data['node_11'][54]]
node_12_data = [nodes_data['node_12'][11], nodes_data['node_12'][33], nodes_data['node_12'][55]]
node_13_data = [nodes_data['node_13'][12], nodes_data['node_13'][34], nodes_data['node_13'][56]]
node_14_data = [nodes_data['node_14'][13], nodes_data['node_14'][35], nodes_data['node_14'][57]]
node_15_data = [nodes_data['node_15'][14], nodes_data['node_15'][36], nodes_data['node_15'][58]]
node_16_data = [nodes_data['node_16'][15], nodes_data['node_16'][37], nodes_data['node_16'][59]]
node_17_data = [nodes_data['node_17'][16], nodes_data['node_17'][38], nodes_data['node_17'][60]]
node_18_data = [nodes_data['node_18'][17], nodes_data['node_18'][39], nodes_data['node_18'][61]]
node_19_data = [nodes_data['node_19'][18], nodes_data['node_19'][40], nodes_data['node_19'][62]]
node_20_data = [nodes_data['node_20'][19], nodes_data['node_20'][41], nodes_data['node_20'][63]]
node_21_data = [nodes_data['node_21'][20], nodes_data['node_21'][42], nodes_data['node_21'][64]]
node_22_data = [nodes_data['node_22'][21], nodes_data['node_22'][43], nodes_data['node_22'][65]]

In [79]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_2_data, ('node_1', 'node_2'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_1, Final weight: 7.395874138425125


In [80]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_5_data, ('node_1', 'node_5'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_5_to_node_1, Final weight: 7.395874138425124


In [81]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_8_data, ('node_1', 'node_8'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_1, Final weight: 7.395874138425126


In [82]:
final_direction, final_weight = analyze_joint_interactions(node_2_data, node_3_data, ('node_2', 'node_3'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_3_to_node_2, Final weight: 7.413877771403063


In [83]:
final_direction, final_weight = analyze_joint_interactions(node_3_data, node_4_data, ('node_3', 'node_4'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_4_to_node_3, Final weight: 7.420040022528913


In [84]:
final_direction, final_weight = analyze_joint_interactions(node_5_data, node_6_data, ('node_5', 'node_6'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_5, Final weight: 7.404110104554736


In [85]:
final_direction, final_weight = analyze_joint_interactions(node_6_data, node_7_data, ('node_6', 'node_7'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_7, Final weight: 7.406204774057298


In [86]:
final_direction, final_weight = analyze_joint_interactions(node_8_data, node_9_data, ('node_8', 'node_9'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_8, Final weight: 7.404821592679698


In [87]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_10_data, ('node_9', 'node_10'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_10, Final weight: 7.405387097318368


In [88]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_15_data, ('node_9', 'node_15'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_15, Final weight: 7.405387097318368


In [89]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_20_data, ('node_9', 'node_20'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_20_to_node_9, Final weight: 7.58340396961296


In [90]:
final_direction, final_weight = analyze_joint_interactions(node_10_data, node_11_data, ('node_10', 'node_11'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_10_to_node_11, Final weight: 7.633300540017764


In [91]:
final_direction, final_weight = analyze_joint_interactions(node_11_data, node_12_data, ('node_11', 'node_12'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_11, Final weight: 7.817321173512099


In [92]:
final_direction, final_weight = analyze_joint_interactions(node_12_data, node_13_data, ('node_12', 'node_13'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_13, Final weight: 7.843179640149443


In [93]:
final_direction, final_weight = analyze_joint_interactions(node_13_data, node_14_data, ('node_13', 'node_14'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_13_to_node_14, Final weight: 7.888197405730156


In [94]:
final_direction, final_weight = analyze_joint_interactions(node_15_data, node_16_data, ('node_15', 'node_16'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_16, Final weight: 7.633300540017619


In [95]:
final_direction, final_weight = analyze_joint_interactions(node_16_data, node_17_data, ('node_16', 'node_17'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_16, Final weight: 7.789234331099532


In [96]:
final_direction, final_weight = analyze_joint_interactions(node_17_data, node_18_data, ('node_17', 'node_18'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_18, Final weight: 7.815193116036021


In [97]:
final_direction, final_weight = analyze_joint_interactions(node_18_data, node_19_data, ('node_18', 'node_19'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_19_to_node_18, Final weight: 7.898498241402666


In [98]:
final_direction, final_weight = analyze_joint_interactions(node_20_data, node_21_data, ('node_20', 'node_21'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_20, Final weight: 7.866747375404851


In [99]:
final_direction, final_weight = analyze_joint_interactions(node_21_data, node_22_data, ('node_21', 'node_22'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_22, Final weight: 7.914118500704991


# Standing Still

In [100]:
nodes_data = {}

for node in range(1, 23):
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44
    node_data = categorized_protective["Standing still"].iloc[:, [x_index, y_index, z_index]]
    nodes_data[f'node_{node}'] = node_data

def analyze_joint_interactions(node_data_1, node_data_2, labels=('node_1', 'node_2')):
    """Analyzes interactions between two nodes and calculates weight based on the direction of influence."""
    directions = []
    kl_divergences = {f'{labels[0]}_to_{labels[1]}': [], f'{labels[1]}_to_{labels[0]}': []}

    for i in range(3):  # For X, Y, Z coordinates
        df = pd.DataFrame({
            labels[0]: node_data_1[i].iloc[:2174],
            labels[1]: node_data_2[i].iloc[:2174]
        })

        pc = PC(df)
        estimated_dag = pc.estimate()
        bn = BayesianModel(estimated_dag.edges())
        bn.fit(df, estimator=MaximumLikelihoodEstimator)

        cpd_for_node_1 = bn.get_cpds(node=labels[0])
        cpd_for_node_2 = bn.get_cpds(node=labels[1])

        p_node_1_given_2 = cpd_for_node_1.values
        p_node_2_given_1 = cpd_for_node_2.values

        kl_divergence_1_to_2 = np.mean(calculate_kl_divergence(p_node_1_given_2, p_node_2_given_1))
        kl_divergence_2_to_1 = np.mean(calculate_kl_divergence(p_node_2_given_1, p_node_1_given_2))

        direction_key = f'{labels[0]}_to_{labels[1]}' if kl_divergence_1_to_2 < kl_divergence_2_to_1 else f'{labels[1]}_to_{labels[0]}'
        directions.append(direction_key)
        kl_divergences[direction_key].append(min(kl_divergence_1_to_2, kl_divergence_2_to_1))

    final_direction = max(set(directions), key=directions.count)
    final_weight = np.mean(kl_divergences[final_direction])

    return final_direction, final_weight

In [101]:
node_1_data = [nodes_data['node_1'][0], nodes_data['node_1'][22], nodes_data['node_1'][44]]
node_2_data = [nodes_data['node_2'][1], nodes_data['node_2'][23], nodes_data['node_2'][45]]
node_3_data = [nodes_data['node_3'][2], nodes_data['node_3'][24], nodes_data['node_3'][46]]
node_4_data = [nodes_data['node_4'][3], nodes_data['node_4'][25], nodes_data['node_4'][47]]
node_5_data = [nodes_data['node_5'][4], nodes_data['node_5'][26], nodes_data['node_5'][48]]
node_6_data = [nodes_data['node_6'][5], nodes_data['node_6'][27], nodes_data['node_6'][49]]
node_7_data = [nodes_data['node_7'][6], nodes_data['node_7'][28], nodes_data['node_7'][50]]
node_8_data = [nodes_data['node_8'][7], nodes_data['node_8'][29], nodes_data['node_8'][51]]
node_9_data = [nodes_data['node_9'][8], nodes_data['node_9'][30], nodes_data['node_9'][52]]
node_10_data = [nodes_data['node_10'][9], nodes_data['node_10'][31], nodes_data['node_10'][53]]
node_11_data = [nodes_data['node_11'][10], nodes_data['node_11'][32], nodes_data['node_11'][54]]
node_12_data = [nodes_data['node_12'][11], nodes_data['node_12'][33], nodes_data['node_12'][55]]
node_13_data = [nodes_data['node_13'][12], nodes_data['node_13'][34], nodes_data['node_13'][56]]
node_14_data = [nodes_data['node_14'][13], nodes_data['node_14'][35], nodes_data['node_14'][57]]
node_15_data = [nodes_data['node_15'][14], nodes_data['node_15'][36], nodes_data['node_15'][58]]
node_16_data = [nodes_data['node_16'][15], nodes_data['node_16'][37], nodes_data['node_16'][59]]
node_17_data = [nodes_data['node_17'][16], nodes_data['node_17'][38], nodes_data['node_17'][60]]
node_18_data = [nodes_data['node_18'][17], nodes_data['node_18'][39], nodes_data['node_18'][61]]
node_19_data = [nodes_data['node_19'][18], nodes_data['node_19'][40], nodes_data['node_19'][62]]
node_20_data = [nodes_data['node_20'][19], nodes_data['node_20'][41], nodes_data['node_20'][63]]
node_21_data = [nodes_data['node_21'][20], nodes_data['node_21'][42], nodes_data['node_21'][64]]
node_22_data = [nodes_data['node_22'][21], nodes_data['node_22'][43], nodes_data['node_22'][65]]

In [102]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_2_data, ('node_1', 'node_2'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_1, Final weight: 7.2245032634381054


In [103]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_5_data, ('node_1', 'node_5'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_5_to_node_1, Final weight: 7.2245032634381054


In [104]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_8_data, ('node_1', 'node_8'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_1, Final weight: 7.2245032634381054


In [105]:
final_direction, final_weight = analyze_joint_interactions(node_2_data, node_3_data, ('node_2', 'node_3'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_3_to_node_2, Final weight: 7.294300887057993


In [106]:
final_direction, final_weight = analyze_joint_interactions(node_3_data, node_4_data, ('node_3', 'node_4'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_4_to_node_3, Final weight: 7.327183686939239


In [107]:
final_direction, final_weight = analyze_joint_interactions(node_5_data, node_6_data, ('node_5', 'node_6'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_5, Final weight: 7.245543525026215


In [108]:
final_direction, final_weight = analyze_joint_interactions(node_6_data, node_7_data, ('node_6', 'node_7'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_7, Final weight: 7.249935071018393


In [109]:
final_direction, final_weight = analyze_joint_interactions(node_8_data, node_9_data, ('node_8', 'node_9'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_8, Final weight: 7.238965576990975


In [110]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_10_data, ('node_9', 'node_10'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_10, Final weight: 7.241289271522018


In [111]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_15_data, ('node_9', 'node_15'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_15, Final weight: 7.24128927152202


In [112]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_20_data, ('node_9', 'node_20'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_20_to_node_9, Final weight: 7.3969086233376045


In [113]:
final_direction, final_weight = analyze_joint_interactions(node_10_data, node_11_data, ('node_10', 'node_11'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_10_to_node_11, Final weight: 7.443102539155066


In [114]:
final_direction, final_weight = analyze_joint_interactions(node_11_data, node_12_data, ('node_11', 'node_12'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_11, Final weight: 7.531921115954883


In [115]:
final_direction, final_weight = analyze_joint_interactions(node_12_data, node_13_data, ('node_12', 'node_13'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_13, Final weight: 7.549831100348546


In [116]:
final_direction, final_weight = analyze_joint_interactions(node_13_data, node_14_data, ('node_13', 'node_14'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_13_to_node_14, Final weight: 7.571353518330341


In [117]:
final_direction, final_weight = analyze_joint_interactions(node_15_data, node_16_data, ('node_15', 'node_16'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_16, Final weight: 7.443102539155149


In [118]:
final_direction, final_weight = analyze_joint_interactions(node_16_data, node_17_data, ('node_16', 'node_17'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_16, Final weight: 7.542558962744671


In [119]:
final_direction, final_weight = analyze_joint_interactions(node_17_data, node_18_data, ('node_17', 'node_18'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_18, Final weight: 7.554903295678663


In [120]:
final_direction, final_weight = analyze_joint_interactions(node_18_data, node_19_data, ('node_18', 'node_19'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_19_to_node_18, Final weight: 7.63382214571962


In [121]:
final_direction, final_weight = analyze_joint_interactions(node_20_data, node_21_data, ('node_20', 'node_21'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_20, Final weight: 7.571889400448566


In [122]:
final_direction, final_weight = analyze_joint_interactions(node_21_data, node_22_data, ('node_21', 'node_22'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_22, Final weight: 7.610718431629988


# Walking

In [132]:
nodes_data = {}

for node in range(1, 23):
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44
    node_data = categorized_protective["Walking"].iloc[:, [x_index, y_index, z_index]]
    nodes_data[f'node_{node}'] = node_data

def analyze_joint_interactions(node_data_1, node_data_2, labels=('node_1', 'node_2')):
    """Analyzes interactions between two nodes and calculates weight based on the direction of influence."""
    directions = []
    kl_divergences = {f'{labels[0]}_to_{labels[1]}': [], f'{labels[1]}_to_{labels[0]}': []}

    for i in range(3):  # For X, Y, Z coordinates
        df = pd.DataFrame({
            labels[0]: node_data_1[i].iloc[:7554],
            labels[1]: node_data_2[i].iloc[:7554]
        })

        pc = PC(df)
        estimated_dag = pc.estimate()
        bn = BayesianModel(estimated_dag.edges())
        bn.fit(df, estimator=MaximumLikelihoodEstimator)

        cpd_for_node_1 = bn.get_cpds(node=labels[0])
        cpd_for_node_2 = bn.get_cpds(node=labels[1])

        p_node_1_given_2 = cpd_for_node_1.values
        p_node_2_given_1 = cpd_for_node_2.values

        kl_divergence_1_to_2 = np.mean(calculate_kl_divergence(p_node_1_given_2, p_node_2_given_1))
        kl_divergence_2_to_1 = np.mean(calculate_kl_divergence(p_node_2_given_1, p_node_1_given_2))

        direction_key = f'{labels[0]}_to_{labels[1]}' if kl_divergence_1_to_2 < kl_divergence_2_to_1 else f'{labels[1]}_to_{labels[0]}'
        directions.append(direction_key)
        kl_divergences[direction_key].append(min(kl_divergence_1_to_2, kl_divergence_2_to_1))

    final_direction = max(set(directions), key=directions.count)
    final_weight = np.mean(kl_divergences[final_direction])

    return final_direction, final_weight

In [133]:
node_1_data = [nodes_data['node_1'][0], nodes_data['node_1'][22], nodes_data['node_1'][44]]
node_2_data = [nodes_data['node_2'][1], nodes_data['node_2'][23], nodes_data['node_2'][45]]
node_3_data = [nodes_data['node_3'][2], nodes_data['node_3'][24], nodes_data['node_3'][46]]
node_4_data = [nodes_data['node_4'][3], nodes_data['node_4'][25], nodes_data['node_4'][47]]
node_5_data = [nodes_data['node_5'][4], nodes_data['node_5'][26], nodes_data['node_5'][48]]
node_6_data = [nodes_data['node_6'][5], nodes_data['node_6'][27], nodes_data['node_6'][49]]
node_7_data = [nodes_data['node_7'][6], nodes_data['node_7'][28], nodes_data['node_7'][50]]
node_8_data = [nodes_data['node_8'][7], nodes_data['node_8'][29], nodes_data['node_8'][51]]
node_9_data = [nodes_data['node_9'][8], nodes_data['node_9'][30], nodes_data['node_9'][52]]
node_10_data = [nodes_data['node_10'][9], nodes_data['node_10'][31], nodes_data['node_10'][53]]
node_11_data = [nodes_data['node_11'][10], nodes_data['node_11'][32], nodes_data['node_11'][54]]
node_12_data = [nodes_data['node_12'][11], nodes_data['node_12'][33], nodes_data['node_12'][55]]
node_13_data = [nodes_data['node_13'][12], nodes_data['node_13'][34], nodes_data['node_13'][56]]
node_14_data = [nodes_data['node_14'][13], nodes_data['node_14'][35], nodes_data['node_14'][57]]
node_15_data = [nodes_data['node_15'][14], nodes_data['node_15'][36], nodes_data['node_15'][58]]
node_16_data = [nodes_data['node_16'][15], nodes_data['node_16'][37], nodes_data['node_16'][59]]
node_17_data = [nodes_data['node_17'][16], nodes_data['node_17'][38], nodes_data['node_17'][60]]
node_18_data = [nodes_data['node_18'][17], nodes_data['node_18'][39], nodes_data['node_18'][61]]
node_19_data = [nodes_data['node_19'][18], nodes_data['node_19'][40], nodes_data['node_19'][62]]
node_20_data = [nodes_data['node_20'][19], nodes_data['node_20'][41], nodes_data['node_20'][63]]
node_21_data = [nodes_data['node_21'][20], nodes_data['node_21'][42], nodes_data['node_21'][64]]
node_22_data = [nodes_data['node_22'][21], nodes_data['node_22'][43], nodes_data['node_22'][65]]

In [134]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_2_data, ('node_1', 'node_2'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [135]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_5_data, ('node_1', 'node_5'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [136]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_8_data, ('node_1', 'node_8'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [137]:
final_direction, final_weight = analyze_joint_interactions(node_2_data, node_3_data, ('node_2', 'node_3'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [138]:
final_direction, final_weight = analyze_joint_interactions(node_3_data, node_4_data, ('node_3', 'node_4'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [139]:
final_direction, final_weight = analyze_joint_interactions(node_5_data, node_6_data, ('node_5', 'node_6'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [140]:
final_direction, final_weight = analyze_joint_interactions(node_6_data, node_7_data, ('node_6', 'node_7'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [141]:
final_direction, final_weight = analyze_joint_interactions(node_8_data, node_9_data, ('node_8', 'node_9'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [142]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_10_data, ('node_9', 'node_10'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [143]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_15_data, ('node_9', 'node_15'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [144]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_20_data, ('node_9', 'node_20'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [145]:
final_direction, final_weight = analyze_joint_interactions(node_10_data, node_11_data, ('node_10', 'node_11'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [146]:
final_direction, final_weight = analyze_joint_interactions(node_11_data, node_12_data, ('node_11', 'node_12'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [147]:
final_direction, final_weight = analyze_joint_interactions(node_12_data, node_13_data, ('node_12', 'node_13'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [148]:
final_direction, final_weight = analyze_joint_interactions(node_13_data, node_14_data, ('node_13', 'node_14'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [149]:
final_direction, final_weight = analyze_joint_interactions(node_15_data, node_16_data, ('node_15', 'node_16'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [150]:
final_direction, final_weight = analyze_joint_interactions(node_16_data, node_17_data, ('node_16', 'node_17'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [151]:
final_direction, final_weight = analyze_joint_interactions(node_17_data, node_18_data, ('node_17', 'node_18'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [152]:
final_direction, final_weight = analyze_joint_interactions(node_18_data, node_19_data, ('node_18', 'node_19'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [153]:
final_direction, final_weight = analyze_joint_interactions(node_20_data, node_21_data, ('node_20', 'node_21'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [154]:
final_direction, final_weight = analyze_joint_interactions(node_21_data, node_22_data, ('node_21', 'node_22'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

# Others

In [155]:
nodes_data = {}

for node in range(1, 23):
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44
    node_data = categorized_protective["Others"].iloc[:, [x_index, y_index, z_index]]
    nodes_data[f'node_{node}'] = node_data

def analyze_joint_interactions(node_data_1, node_data_2, labels=('node_1', 'node_2')):
    """Analyzes interactions between two nodes and calculates weight based on the direction of influence."""
    directions = []
    kl_divergences = {f'{labels[0]}_to_{labels[1]}': [], f'{labels[1]}_to_{labels[0]}': []}

    for i in range(3):  # For X, Y, Z coordinates
        df = pd.DataFrame({
            labels[0]: node_data_1[i].iloc[:16135],
            labels[1]: node_data_2[i].iloc[:16135]
        })

        pc = PC(df)
        estimated_dag = pc.estimate()
        bn = BayesianModel(estimated_dag.edges())
        bn.fit(df, estimator=MaximumLikelihoodEstimator)

        cpd_for_node_1 = bn.get_cpds(node=labels[0])
        cpd_for_node_2 = bn.get_cpds(node=labels[1])

        p_node_1_given_2 = cpd_for_node_1.values
        p_node_2_given_1 = cpd_for_node_2.values

        kl_divergence_1_to_2 = np.mean(calculate_kl_divergence(p_node_1_given_2, p_node_2_given_1))
        kl_divergence_2_to_1 = np.mean(calculate_kl_divergence(p_node_2_given_1, p_node_1_given_2))

        direction_key = f'{labels[0]}_to_{labels[1]}' if kl_divergence_1_to_2 < kl_divergence_2_to_1 else f'{labels[1]}_to_{labels[0]}'
        directions.append(direction_key)
        kl_divergences[direction_key].append(min(kl_divergence_1_to_2, kl_divergence_2_to_1))

    final_direction = max(set(directions), key=directions.count)
    final_weight = np.mean(kl_divergences[final_direction])

    return final_direction, final_weight

In [156]:
node_1_data = [nodes_data['node_1'][0], nodes_data['node_1'][22], nodes_data['node_1'][44]]
node_2_data = [nodes_data['node_2'][1], nodes_data['node_2'][23], nodes_data['node_2'][45]]
node_3_data = [nodes_data['node_3'][2], nodes_data['node_3'][24], nodes_data['node_3'][46]]
node_4_data = [nodes_data['node_4'][3], nodes_data['node_4'][25], nodes_data['node_4'][47]]
node_5_data = [nodes_data['node_5'][4], nodes_data['node_5'][26], nodes_data['node_5'][48]]
node_6_data = [nodes_data['node_6'][5], nodes_data['node_6'][27], nodes_data['node_6'][49]]
node_7_data = [nodes_data['node_7'][6], nodes_data['node_7'][28], nodes_data['node_7'][50]]
node_8_data = [nodes_data['node_8'][7], nodes_data['node_8'][29], nodes_data['node_8'][51]]
node_9_data = [nodes_data['node_9'][8], nodes_data['node_9'][30], nodes_data['node_9'][52]]
node_10_data = [nodes_data['node_10'][9], nodes_data['node_10'][31], nodes_data['node_10'][53]]
node_11_data = [nodes_data['node_11'][10], nodes_data['node_11'][32], nodes_data['node_11'][54]]
node_12_data = [nodes_data['node_12'][11], nodes_data['node_12'][33], nodes_data['node_12'][55]]
node_13_data = [nodes_data['node_13'][12], nodes_data['node_13'][34], nodes_data['node_13'][56]]
node_14_data = [nodes_data['node_14'][13], nodes_data['node_14'][35], nodes_data['node_14'][57]]
node_15_data = [nodes_data['node_15'][14], nodes_data['node_15'][36], nodes_data['node_15'][58]]
node_16_data = [nodes_data['node_16'][15], nodes_data['node_16'][37], nodes_data['node_16'][59]]
node_17_data = [nodes_data['node_17'][16], nodes_data['node_17'][38], nodes_data['node_17'][60]]
node_18_data = [nodes_data['node_18'][17], nodes_data['node_18'][39], nodes_data['node_18'][61]]
node_19_data = [nodes_data['node_19'][18], nodes_data['node_19'][40], nodes_data['node_19'][62]]
node_20_data = [nodes_data['node_20'][19], nodes_data['node_20'][41], nodes_data['node_20'][63]]
node_21_data = [nodes_data['node_21'][20], nodes_data['node_21'][42], nodes_data['node_21'][64]]
node_22_data = [nodes_data['node_22'][21], nodes_data['node_22'][43], nodes_data['node_22'][65]]

In [157]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_2_data, ('node_1', 'node_2'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_1, Final weight: 9.627937141368667


In [158]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_5_data, ('node_1', 'node_5'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_5_to_node_1, Final weight: 9.627937141368669


In [159]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_8_data, ('node_1', 'node_8'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_1, Final weight: 9.62793714136867


In [160]:
final_direction, final_weight = analyze_joint_interactions(node_2_data, node_3_data, ('node_2', 'node_3'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_3_to_node_2, Final weight: 9.639712341524453


In [161]:
final_direction, final_weight = analyze_joint_interactions(node_3_data, node_4_data, ('node_3', 'node_4'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_4_to_node_3, Final weight: 9.643181554584473


In [162]:
final_direction, final_weight = analyze_joint_interactions(node_5_data, node_6_data, ('node_5', 'node_6'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_5, Final weight: 9.63930650385141


In [163]:
final_direction, final_weight = analyze_joint_interactions(node_6_data, node_7_data, ('node_6', 'node_7'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_7, Final weight: 9.641443794499835


In [164]:
final_direction, final_weight = analyze_joint_interactions(node_8_data, node_9_data, ('node_8', 'node_9'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_8, Final weight: 9.63619757957026


In [165]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_10_data, ('node_9', 'node_10'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_10, Final weight: 9.636387022946275


In [166]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_15_data, ('node_9', 'node_15'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_15, Final weight: 9.636387022946279


In [167]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_20_data, ('node_9', 'node_20'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_20_to_node_9, Final weight: 9.652017336042524


In [168]:
final_direction, final_weight = analyze_joint_interactions(node_10_data, node_11_data, ('node_10', 'node_11'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_10_to_node_11, Final weight: 9.659827888870316


In [169]:
final_direction, final_weight = analyze_joint_interactions(node_11_data, node_12_data, ('node_11', 'node_12'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_11, Final weight: 9.676638409601098


In [170]:
final_direction, final_weight = analyze_joint_interactions(node_12_data, node_13_data, ('node_12', 'node_13'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_13, Final weight: 9.67923935459599


In [171]:
final_direction, final_weight = analyze_joint_interactions(node_13_data, node_14_data, ('node_13', 'node_14'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_13_to_node_14, Final weight: 9.683091623523259


In [172]:
final_direction, final_weight = analyze_joint_interactions(node_15_data, node_16_data, ('node_15', 'node_16'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_16, Final weight: 9.659827888872266


In [173]:
final_direction, final_weight = analyze_joint_interactions(node_16_data, node_17_data, ('node_16', 'node_17'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_16, Final weight: 9.674829512499429


In [174]:
final_direction, final_weight = analyze_joint_interactions(node_17_data, node_18_data, ('node_17', 'node_18'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_18, Final weight: 9.677716442229832


In [175]:
final_direction, final_weight = analyze_joint_interactions(node_18_data, node_19_data, ('node_18', 'node_19'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_19_to_node_18, Final weight: 9.68467856821868


In [176]:
final_direction, final_weight = analyze_joint_interactions(node_20_data, node_21_data, ('node_20', 'node_21'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_20, Final weight: 9.672940846251281


In [177]:
final_direction, final_weight = analyze_joint_interactions(node_21_data, node_22_data, ('node_21', 'node_22'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_22, Final weight: 9.67859444210544
